In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
path = './clothing-dataset-small/train/t-shirt/5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg'
load_img(path)

In [ ]:
load_img(path, target_size=(150, 150))

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

In [ ]:
img = load_img(path, target_size=(299, 299))
X = np.array(img)
X = np.array([X])

X.shape

In [ ]:
X

In [ ]:
X = preprocess_input(X)
X

In [ ]:
model = Xception(
    weights="imagenet",
    input_shape=(299, 299, 3)
)

In [ ]:
y_pred = model.predict(X)

In [ ]:
from tensorflow.keras.applications.xception import decode_predictions

In [ ]:
decode_predictions(y_pred)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10.0,
    zoom_range=0.1,
    horizontal_flip=True,  
)

train_ds = train_gen.flow_from_directory(
    'clothing-dataset-small/train/',
    seed=1,
    target_size=(299, 299),
    batch_size=32
)

In [ ]:
val_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

val_ds = train_gen.flow_from_directory(
    'clothing-dataset-small/validation/',
    seed=1,
    target_size=(299, 299),
    batch_size=32
)

In [ ]:
base_model = Xception(
    weights='imagenet',
    input_shape=(299, 299, 3),
    include_top=False
)

base_model.trainable = False

inputs = keras.Input(shape=(299, 299, 3))

base = base_model(inputs, training=False)
pooling_function = keras.layers.GlobalAveragePooling2D()

vector = pooling_function(base)

inner = keras.layers.Dense(200, activation='relu')(vector)

dropout = keras.layers.Dropout(0.3)(inner)

output = keras.layers.Dense(10)(dropout)

model = keras.Model(inputs, output)

In [ ]:
learning_rate = 0.001 

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        'model_v3_{epoch}_{val_accuracy:.3f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )
]

In [ ]:
model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=callbacks)